In [ ]:
import folium
import pandas as pd

def categoria_saffir_simpson(vento_kmh):
    """Restituisce una sfumatura di blu in base alla scala Saffir-Simpson."""
    if vento_kmh < 119:        
        return '#d0e6f5'        
    elif 119 <= vento_kmh <= 153:  
        return '#99c2eb'
    elif 154 <= vento_kmh <= 177:  
        return '#4d94db'
    elif 178 <= vento_kmh <= 208: 
        return '#1f78b4'
    elif 209 <= vento_kmh <= 251:  
        return '#0b5394'
    else: 
        return '#062c53'        

def crea_mappa():
    m = folium.Map(location=[15, 125], zoom_start=4, tiles='CartoDB positron')

    colori_ciclone = {'Katrina': '#156082', 'Nargis': '#0b3d6e', 'Doksuri': '#031f3d'}

    df = pd.read_csv('../data/cicloni_traiettorie.csv')
    cicloni_grouped = df.groupby('ciclone')

    for nome, dati in cicloni_grouped:
        coords = dati[['latitudine', 'longitudine']].values.tolist()
        folium.PolyLine(coords, color=colori_ciclone.get(nome, '#156082'),
                        weight=3, opacity=0.7, tooltip=nome).add_to(m)

        for _, punto in dati.iterrows():
            colore = categoria_saffir_simpson(punto['vento_kmh'])
            folium.Circle(
                location=[punto['latitudine'], punto['longitudine']],
                radius=punto['width'] * 1000 / 2,
                color=colore,
                fill=True,
                fill_opacity=0.5,
                tooltip=f"Vento: {punto['vento_kmh']} km/h"
            ).add_to(m)

    legend_html = '''
     <div style="position: fixed; bottom: 50px; left: 50px; width: 350px; height: 290px;
     background-color: white; border:2px solid grey; z-index:9999; font-size:22px;
     padding: 10px;">
     <b>Legenda categoria Saffir-Simpson</b><br>
     <span style="color:#d0e6f5;">■</span>&nbsp;Non ciclone (<119 km/h)<br>
     <span style="color:#99c2eb;">■</span>&nbsp;Categoria 1 (119–153 km/h)<br>
     <span style="color:#4d94db;">■</span>&nbsp;Categoria 2 (154–177 km/h)<br>
     <span style="color:#1f78b4;">■</span>&nbsp;Categoria 3 (178–208 km/h)<br>
     <span style="color:#0b5394;">■</span>&nbsp;Categoria 4 (209–251 km/h)<br>
     <span style="color:#062c53;">■</span>&nbsp;Categoria 5 (≥252 km/h)<br>
     </div>
    '''

    titolo_html = '''
    <div style="position: fixed; top: 10px; left: 50%; transform: translateX(-50%);
                background-color: rgba(255,255,255,0.9); padding: 10px 20px;
                border: 2px solid #555; border-radius: 8px;
                font-size: 28px; font-weight: bold; z-index: 9999;">
        Traiettorie Cicloni Tropicali
    </div>
    '''

    m.get_root().html.add_child(folium.Element(titolo_html))
    m.get_root().html.add_child(folium.Element(legend_html))

    return m

mappa = crea_mappa()
mappa.save('Traiettoria_cicloni_tropicali_casi.html')
